In [16]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
import cv2
import os
from ultralytics import YOLO
import time, json



In [17]:
YOLO_MODELS = [
    'yolov5m6u.pt',  # Ultra-real-time optimized
    'yolov8m.pt',    # Balanced accuracy/speed
    'yolov8x.pt',    # High accuracy
    'yolov9e.pt'     # Best Performer (Target Model)
]

In [18]:
with open('models.json', 'r') as f:
    model_paths = json.load(f)
    yolo5m6u = model_paths["models"]["yolo5m6u"]
    yolo8m = model_paths["models"]["yolo8m"]
    yolo8x = model_paths["models"]["yolo8x"]
    yolo9e = model_paths["models"]["yolo9e"]

#models = [yolo5m6u, yolo8m, yolo8x, yolo9e]
models = [r"C:\Users\Florian Caspar\Desktop\Desly\DefectClassification-main\fdm_manual_training\yolov8_final_run\weights\best.pt"]
image = r"C:\Users\Florian Caspar\Desktop\Desly\FDM-3D-Printing-Defect-Dataset\data\Stringing\Image_20240118113025171.jpg"
img = cv2.imread(image)
height, width, channels = img.shape

print(f"Width: {width}, Height: {height}, Channels: {channels}")

resized_img = cv2.resize(img, (500, 300))
h, w, c = resized_img.shape
print(f"New Dimensions - Width: {w}, Height: {h}")

Width: 3072, Height: 2048, Channels: 3
New Dimensions - Width: 500, Height: 300


With SAHI


In [19]:
# List to store timing results for comparison later
timing_report = []

for model in models:
    model_name = model.split("\\")[-1].removesuffix(".pt")
    print(f"--- Processing: {model_name} ---")

    detection_model = AutoDetectionModel.from_pretrained(
        model_type='ultralytics',
        model_path=model,
        confidence_threshold=0.66,  #results with 66 percent surety
        device='cpu' # "cuda" for gpu
    )


    start_time = time.perf_counter()

    results = get_sliced_prediction(
        resized_img,
        detection_model,
        slice_height=100,
        slice_width=100,
        overlap_height_ratio=0.1,
        overlap_width_ratio=0.1
    )

    end_time = time.perf_counter()
    elapsed_time = end_time - start_time

    print(f"Inference Time: {elapsed_time:.4f} seconds")
    
    timing_report.append({"model": model_name, "time": elapsed_time})

    results.export_visuals(
        export_dir="demo_inference_results/",
        file_name=f"Sahi_prediction_{model_name}"
    )


    for prediction in results.object_prediction_list:
        class_name = prediction.category.name
        score = prediction.score.value
        print(f" - {class_name}: {score:.2%}")

# --- Summary ---
print("\n--- Performance Summary ---")
for entry in timing_report:
    print(f"{entry['model']}: {entry['time']:.4f}s")

--- Processing: best ---
Performing prediction on 24 slices.
Inference Time: 3.1353 seconds

--- Performance Summary ---
best: 3.1353s


DIRECT INFERENCE

In [20]:
os.makedirs("demo_inference_results", exist_ok=True)

timing_report = []

for model_path in models:
    model_name = model_path.split("\\")[-1].removesuffix(".pt")
    print(f"--- Processing: {model_name} ---")

  
    model = YOLO(model_path)

    start_time = time.perf_counter()

    
    results = model.predict(
        source=resized_img, 
        conf=0.66,      # 66% confidence threshold
        device='cpu',   # or '0' for GPU
        verbose=True
    )

    end_time = time.perf_counter()
    elapsed_time = end_time - start_time

    print(f"Inference Time: {elapsed_time:.4f} seconds")
    timing_report.append({"model": model_name, "time": elapsed_time})

   
    result = results[0]

    plotted_img = result.plot()
    save_path = os.path.join("demo_inference_results", f"Direct_prediction_{model_name}.jpg")
    cv2.imwrite(save_path, plotted_img)

    if len(result.boxes) > 0:
        for box in result.boxes:
            # box.cls is a tensor, we convert to int
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            # box.conf is a tensor, we convert to float
            score = float(box.conf[0])
            print(f" - {class_name}: {score:.2%}")
        print("")
    else:
        print(" - No detections found.")

# --- Summary ---
print("\n--- Performance Summary ---")
for entry in timing_report:
    print(f"{entry['model']}: {entry['time']:.4f}s")

--- Processing: best ---

0: 384x640 (no detections), 115.4ms
Speed: 1.4ms preprocess, 115.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Inference Time: 0.1761 seconds
 - No detections found.

--- Performance Summary ---
best: 0.1761s


FINETUNED MODEL

In [27]:
import cv2
from ultralytics import YOLO

# ==========================================
# CONFIGURATION
# ==========================================
# 1. Path to your trained weights
# (Adjust this path to where your 'best.pt' is actually located)
MODEL_PATH = r"C:\Users\Florian Caspar\Desktop\Desly\DefectClassification-main\fdm_manual_training\yolov8_final_run\weights\best.pt"

# 2. Path to the image you want to test
IMAGE_PATH = r"C:\Users\Florian Caspar\Desktop\Desly\FDM-3D-Printing-Defect-Dataset\data\Cracking\Image_20231128202706999.jpg"

# ==========================================
# EXECUTION
# ==========================================
def run_inference():
    # 1. Load the Custom Model
    print(f"Loading model from: {MODEL_PATH}")
    model = YOLO(MODEL_PATH)

    # 2. Run Prediction
    # conf=0.25: Only show detections with >25% confidence
    # save=True: Saves the annotated image to 'runs/detect/predict'
    results = model.predict(IMAGE_PATH, conf=0.25, save=True)

    # 3. Process & Print Results
    result = results[0]  # We only processed one image
    
    print("\n" + "="*30)
    print("      DETECTION RESULTS")
    print("="*30)

    if len(result.boxes) == 0:
        print("No defects detected.")
    else:
        for box in result.boxes:
            # Extract info
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            confidence = float(box.conf[0])
            
            # Print to console
            print(f"-> Found: {class_name} ({confidence:.1%} confidence)")
            
            # (Optional) Get coordinates if you need them for automation
            # x1, y1, x2, y2 = box.xyxy[0].tolist()
            # print(f"   Location: {x1:.0f}, {y1:.0f}, {x2:.0f}, {y2:.0f}")

    # 4. Display the Image (Optional)
    # result.plot() creates a BGR numpy array with boxes drawn
    annotated_frame = result.plot()
    small_frame = cv2.resize(annotated_frame, (0, 0), fx=0.3, fy=0.3)
    
    cv2.imshow("YOLO Inference", small_frame)
    print("\nPress any key to close the image window...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Verify the model path exists before running to avoid confusion
import os
if not os.path.exists(MODEL_PATH):
    print(f"ERROR: Could not find model file at: {MODEL_PATH}")
    print("Please check the folder path.")
else:
    run_inference()

Loading model from: C:\Users\Florian Caspar\Desktop\Desly\DefectClassification-main\fdm_manual_training\yolov8_final_run\weights\best.pt

image 1/1 C:\Users\Florian Caspar\Desktop\Desly\FDM-3D-Printing-Defect-Dataset\data\Cracking\Image_20231128202706999.jpg: 448x640 1 cracking, 142.3ms
Speed: 1.7ms preprocess, 142.3ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)
Results saved to C:\Users\Florian Caspar\Desktop\Desly\DefectClassification-main\runs\detect\predict6

      DETECTION RESULTS
-> Found: cracking (26.1% confidence)

Press any key to close the image window...
